In [1]:
!pip install requests

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install pandas

In [4]:
!pip install lxml

In [5]:
!pip install selenium

In [23]:
!pip install python-dotenv

In [9]:
years = range(1994, 2024)

In [10]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [ ]:
import requests

for year in years: 
    url = url_start.format(year)
    data = requests.get(url)
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)


In [13]:
# import pandas as pd
# from io import StringIO
# from bs4 import BeautifulSoup

# with open("mvp/1994.html") as f:
#     page = f.read()

# soup = BeautifulSoup(page, "html.parser")

# # Removing extra header
# soup.find('tr', class_="over_header").decompose()

# # Removing everything else from the page outside of the mvp table we want
# mvp_table = soup.find(id="mvp")

# mvp_1994 = pd.read_html(StringIO(str(mvp_table)))

# # Displays single data frame
# mvp_1994[0]

In [11]:
# Parsing all of the tables
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

dfs = []
for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find(id="mvp")
    mvp = pd.read_html(StringIO(str(mvp_table)))[0]

    # Adding a year column
    mvp["Year"] = year
    
    dfs.append(mvp)

In [12]:
# Combining all of the dataframes
mvps = pd.concat(dfs)

In [16]:
mvps.to_csv("mvps.csv")

In [13]:
# Looking at stats for all players in the same year range

player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
# This page uses javascript to render all of the rows because of this we need to use selenium

In [4]:
from dotenv import load_dotenv
import os
from selenium import webdriver

load_dotenv()

True

In [5]:
driver_url = os.getenv("DRIVER_PATH")
driver = webdriver.Chrome(executable_path=driver_url)

C:\Users\Caleb\AppData\Local\Temp\ipykernel_44704\2442837788.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_url)


In [ ]:
import time

for year in years:
    # Telling the driver to get all of the data from the page
    url = player_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1, 10000)")
    
    # Giving enough time to execute the commands
    time.sleep(2)
    
    # Creating player stats html files
    html = driver.page_source
    with open("player/{}.html".format(year), "w+") as f:
        f.write(html)

In [ ]:
# Removing extra headers again
dfs = []
for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="thead").decompose()
        player_table = soup.find(id="per_game_stats")
        player = pd.read_html(str(player_table))[0]
        player["Year"] = year
        dfs.append(player)
        
players = pd.concat(dfs)

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
0,1,David Robinson,28,SAS,C,80,80,40.5,10.5,20.7,...,7.7,10.7,4.8,1.7,3.3,3.2,2.9,29.8,"MVP-2,DPOY-2,AS,NBA2",1994
1,2,Shaquille O'Neal,21,ORL,C,81,81,39.8,11.8,19.6,...,8.5,13.2,2.4,0.9,2.9,2.7,3.5,29.3,"MVP-4,AS,NBA3",1994
2,3,Hakeem Olajuwon,31,HOU,C,80,80,41.0,11.2,21.2,...,9.1,11.9,3.6,1.6,3.7,3.4,3.6,27.3,"MVP-1,DPOY-1,AS,NBA1",1994
3,4,Dominique Wilkins,34,2TM,SF,74,74,35.6,9.4,21.5,...,4.0,6.5,2.3,1.2,0.4,2.3,1.7,26.0,"MVP-11,AS,NBA3",1994
4,4,Dominique Wilkins,34,ATL,SF,49,49,34.4,8.8,20.3,...,3.8,6.2,2.3,1.3,0.4,2.4,1.8,24.4,NaN,1994


In [15]:
players.to_csv("players.csv")

In [18]:
# Downloading team data
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

# Using division standings because conference standings table uses javascript
for year in years:
    url = team_stats_url.format(year)
    data = requests.get(url)
    with open("team/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [20]:
# Parsing team data
dfs = []
for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="thead").decompose()
    team_table = soup.find(id="divs_standings_E")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    
    # Assigning whatever is listed under the Conference column to the Team column we are creating to represent the team name
    team["Team"] = team["Eastern Conference"]
    
    # Removing unused column
    del team["Eastern Conference"]
    dfs.append(team)
    
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="thead").decompose()
    team_table = soup.find(id="divs_standings_W")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    dfs.append(team)

In [21]:
teams = pd.concat(dfs)

In [22]:
teams.to_csv("teams.csv")